In [ ]:
import cv2
import glob
import os
import imutils
import numpy as np
import matplotlib.pyplot as plt
import stitching
from stitching.cropper import Cropper

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
video_path_left = os.path.join("/media/jakki/Qumran/Videos/meow_tmp/wb8y5mj8.mp4")
video_path_right = os.path.join("/media/jakki/Qumran/Videos/meow_tmp/t8pxz3yw.mp4")

In [ ]:
capture_left = cv2.VideoCapture(video_path_left)
_, image_left = capture_left.read()

capture_right = cv2.VideoCapture(video_path_right)
_, image_right = capture_right.read()

In [ ]:
image_left.shape

In [ ]:
best_matrix = "camera_1_checkerboard_8x10_20x20_slow"

calibration_data_folder = os.path.join(DATA_FOLDER, "camera_calibration", "go_pro_5_hero_black")

intrinsic_matrix = np.array([
    [1039.788495388855, 0, 961.4873458747151],
    [0, 1045.3324681263186, 532.5995495852319], 
    [0, 0, 1]
])

distortion = np.array([
    -0.2830088858305849, 0.1353401508767815, -0.0005040870884686496, -0.000176582493717214
])

In [ ]:
h, w = 1080, 1920
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w, h), 1, (w, h))

undistorted_image_left = cv2.undistort(image_left, intrinsic_matrix, distortion, None, undistorted_matrix)

plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image_left)
plt.subplot(1,2,2)
plt.imshow(undistorted_image_left)
plt.show()

In [ ]:
h, w = 1080, 1920
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w, h), 1, (w, h))

undistorted_image_right = cv2.undistort(image_right, intrinsic_matrix, distortion, None, undistorted_matrix)

plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image_right)
plt.subplot(1,2,2)
plt.imshow(undistorted_image_right)
plt.show()

In [ ]:
# Fisheye, affine, stereographic and portrait projections don't work

projections = [
    "spherical",
    "plane",
    "affine",
    "cylindrical",
    "compressedPlaneA2B1",
    "compressedPlaneA1.5B1",
    "paniniA2B1",
    "paniniA1.5B1",
    "mercator",
    "transverseMercator"
]

for proj in projections:
    try:
        stitcher = stitching.Stitcher(warper_type=proj, crop=None, wave_correct_kind="horiz", blend_strength=20, try_use_gpu=True)
        panorama = stitcher.stitch([undistorted_image_left, undistorted_image_right])
    except Exception as exc:
        print(exc)
        continue
        
    plt.figure(figsize=(32,32))
    plt.title(f"Project: {proj}")
    plt.imshow(panorama)
    plt.show()

In [ ]:
stitcher = stitching.Stitcher(warper_type="transverseMercator", crop=None, wave_correct_kind="horiz", blend_strength=20, try_use_gpu=True)
panorama = stitcher.stitch([undistorted_image_left, undistorted_image_right])
panorama = imutils.rotate(panorama, 3)
cropped_pano = panorama[150:1000, 100:3000]
plt.figure(figsize=(30,30))
plt.imshow(cropped_pano)
plt.show()

In [ ]:
best_matrix = "camera_1_checkerboard_8x10_20x20_slow"

calibration_data_folder = os.path.join(DATA_FOLDER, "camera_calibration", "go_pro_5_hero_black")

intrinsic_matrix = np.load(
    os.path.join(
        calibration_data_folder, "camera_1", f"{best_matrix}_intrinsic_matrix.npy"
    )
)
distortion = np.load(
    os.path.join(
        calibration_data_folder, "camera_1", f"{best_matrix}_camera_distortion.npy"
    )
)

In [ ]:
h, w = 1080, 1920
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w, h), 1, (w, h))

x, y, w, h = roi
undistorted_image_right2 = cv2.undistort(image_right, intrinsic_matrix, distortion, None, undistorted_matrix)
#undistorted_image_right2 = undistorted_image_right2[y:y+h, x:x+w]

plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image_right)
plt.subplot(1,2,2)
plt.imshow(undistorted_image_right2)
plt.show()